# MetricGAN

Представленная в статье [MetricGAN: Generative Adversarial Networks based Black-box Metric Scores
Optimization for Speech Enhancement](https://arxiv.org/pdf/1905.04874.pdf) архитектура решает проблему Speech Enhancement, то есть улучшения речи, а именно повышения ее разборчивости и качества.


В качестве оценки для модели были использованы две метрики *PESQ(perceptual evaluation of speech quality)* и *STOI(short-time objective intelligibility)*. Основным предположением авторов данной статьи было то что использующиеся в данный момент GAN-based подходы для данной задачи не достигают достаточной эффективности потому что то как дискриминатор решает настоящий ли пример или поддельный может никаким образом не относиться к желаемой метрике и соответственно значительно ее не улучшать. В статье авторы предлагают решение для данной проблемы показывая, что их модель превосходит предыдущие на нескольких датасетах и на совершенно различных метриках. 


Предлагаемое ими решение заключается в том, что вместо обычной бинарной классификации на реальный объект или подделку дискриминатор будет пытаться предсказать значение метрики для полученного им образца. Это решение выглядит очень элегантно и как показыали их экспериметы обходит по результатам предыдущие подходы. Одна из его прелестей в том, что модель не имеет никакого предстваления о том как выглядит или считается используемая метрика и поэтому мы можем использовать данный подход для любой метрики независимо от того дифференцируем ли она или нет.


Оптимизируемая функция для дискриминатора из статьи
$$L_D=\mathbb{E}_{x,y}[(D(y,y) - Q(y,y))^2 + (D(G(x), y) - Q(G(x), y))^2]$$


В функции выше $Q$ - это значение нашей метрики сжатое в отрезок $[0,1]$, а $y$ вторым аргуметом дискриминатора это передаваемый ему condition(как condition использовали чистый сигнал потому что хотели чтобы дискриминатор имел схожее с метрикой поведение).


Оптимизируемая функция же для тренировки генератора выглядела схожим образом:
$$L_G=\mathbb{E}_{x}[(D(G(x), y) - s)^2]$$
где $s$ - значение метрики, которое мы хотим получить. Обычно это 1, но авторы подмечают интересный факт, что можно тренировать модели целенаправленно ухудшать метрики например выставляя значение около $0.2$.


### Мой эксперимент

Выполнить данный эксперимент я решил с помощью фреймворка PyTorch.

К сожалению моих ресурсов было недостаточно для того чтобы воссоздать все условия для обучения модели из статьи, даже не смотря на то что учились они на датасете детского размера TIMIT. В эксперименте проведенном авторами статьи модель училась в течение 200 итераций общего пайплайна. Для того чтобы выполнить все это на своей машине мне пришлось урезать размер данных для обучения до 300 примеров($\frac{1}{10}$ от всего датасета), и снизить количество итераций до 50. И даже в этом случае моя модель обучалась 7 часов. На мой взгляд именно это сказалось на том, что моя модель в результате показала результаты хуже чем в статье.


Ниже я приведу график обучения модели. К сожалению мне не хватило врмени чтобы выполнять все обучение именно в notebook'е. Поэтому весь код можно найти в приложенном репозитории.


![image](info/history.jpg)


На картинке представленно обучение модели сжатой метрике PESQ, то есть верхний достигнутый ею порог в PESQ будет равен 1.3.


Замеры происходили на каждой 10 итерации обучения, поэтому они довольно редкие, но даже здесь видно что возможно произошло переобучение(опять же, виню размер данных, зная сколько часов речи нужно для обучения обычных моделей работающих с голосом 1 час кажется абсолютно ничего не значащей цифрой).
